In [2]:
import math
import numpy as np
import pandas as pd

def normal_cdf(x):
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

def diebold_mariano(e1, e2, h=1, power=1):
    
    e1, e2 = np.asarray(e1), np.asarray(e2)
    d = np.abs(e1)**power - np.abs(e2)**power
    T = len(d)
    mean_d = np.mean(d)

    # Newey–West variance with truncation lag h-1
    var_d = np.var(d, ddof=1)
    for lag in range(1, h):
        cov = np.cov(d[:-lag], d[lag:])[0, 1]
        var_d += 2 * (1 - lag / h) * cov
    var_d /= T

    dm_stat = mean_d / math.sqrt(var_d)
    p_value = 2 * (1 - normal_cdf(abs(dm_stat)))
    return dm_stat, p_value

dm_input_long = pd.read_csv("dm_yhat_long.csv", parse_dates=["date"])
actual_df = pd.read_csv("../data/data_main.csv", parse_dates=["date"])
dm_input_long = dm_input_long.merge(
     actual_df.rename(columns={"log_total_dollar_volume": "actual"}),
     on="date",
     how="inner",
)

dm_input_long["use_exog"] = dm_input_long["model"].str.endswith("X")
dm_input_long["base_model"] = dm_input_long["model"].str.rstrip("X")

tests = []

for split in sorted(dm_input_long["split"].unique()):
    df_split = dm_input_long[dm_input_long["split"] == split]

    for base in sorted(df_split["base_model"].unique()):
        df_family = df_split[df_split["base_model"] == base]

        # Non-exogenous version(s)
        df_no_exog = df_family[df_family["use_exog"] == False].sort_values("date")
        # Exogenous version(s)
        df_with_exog = df_family[df_family["use_exog"] == True].sort_values("date")

        # Need both to compare
        if df_no_exog.empty or df_with_exog.empty:
            continue

        # If there are multiple variants, you might have multiple rows per date;
        # here I just take the first model of each group.
        model_no_exog  = df_no_exog["model"].iloc[0]
        model_with_exog = df_with_exog["model"].iloc[0]

        df_no_exog_1 = df_no_exog[df_no_exog["model"] == model_no_exog]
        df_with_exog_1 = df_with_exog[df_with_exog["model"] == model_with_exog]

        merged = pd.merge(
            df_no_exog_1[["date", "actual", "yhat"]].rename(
                columns={"yhat": "yhat_no", "actual": "actual_no"}
            ),
            df_with_exog_1[["date", "actual", "yhat"]].rename(
                columns={"yhat": "yhat_ex", "actual": "actual_ex"}
            ),
            on="date",
            how="inner",
        )

        if merged.empty:
            continue

        # actuals should be identical in both; take one
        actual = merged["actual_no"]

        e_no  = actual - merged["yhat_no"]
        e_exo = actual - merged["yhat_ex"]

        # 1-step ahead forecasts, so h = 1; using absolute-error loss (power=1).
        dm_stat, pval = diebold_mariano(e_no, e_exo, h=1, power=1)

        if pval < 0.05:
            winner = "with_exog" if dm_stat > 0 else "no_exog"
        else:
            winner = "no significant diff"

        tests.append({
            "split": split,
            "base_model": base,              # e.g. "AR", "MA", "ARMA"
            "model_no_exog":  model_no_exog,
            "model_with_exog": model_with_exog,
            "dm_stat": dm_stat,
            "p_value": pval,
            "better_model": winner,
            "n_obs": len(merged),
        })

dm_results = pd.DataFrame(tests).sort_values(["split", "base_model"])
print(dm_results)

     split base_model model_no_exog model_with_exog   dm_stat   p_value  \
0  Split 1         AR            AR             ARX  0.328657  0.742415   
1  Split 1      ARIMA         ARIMA          ARIMAX -0.046693  0.962758   
2  Split 1         MA            MA             MAX -0.748717  0.454028   
3  Split 2         AR            AR             ARX  0.111860  0.910934   
4  Split 2      ARIMA         ARIMA          ARIMAX -0.094597  0.924635   
5  Split 2         MA            MA             MAX  0.010866  0.991330   

          better_model  n_obs  
0  no significant diff    502  
1  no significant diff    502  
2  no significant diff    502  
3  no significant diff    502  
4  no significant diff    502  
5  no significant diff    502  
